<center>

<font size="5">MGMTMSA 408: Operations Analytics</font>

</center>


<center>

<font size="5">Assignment 2: Constraint Generation and Assortment Optimization</font>

</center>


<div style="display: flex; justify-content: space-between;">
    <div style="text-align: left;">
        Due on BruinLearn by 1:00pm on May 06, 2024.
    </div>
    <div style="text-align: right;">
        Arnav Garg (906310841) <br>
    </div>
</div>


**<font size="5">1. Learning Coffee Preferences</font>**

In this problem, we will be using column / constraint generation to learn about customer preferences
through a discrete choice model. In a discrete choice model, there are n distinct products, indexed
from 1 to n, as well as a dummy product 0, which represents the outside or no-purchase option
(i.e., the customer chooses not to buy any of the products from the assortment). When a customer
makes a decision, they choose a product from within the assortment or they choose the no-purchase
option. The specific type of discrete choice model we will use is the ranking-based choice model,
which will be explained in more detail in Part 1 and Part 2 of this problem.
The data set that we have consists of different assortments of coffee brands, which are listed
below. For each assortment, we observe how many customers purchased each of those brands, as
well as how many customers did not purchase anything when they considered the assortment. The
data will be described in more detail in Part 3.

Product # | Brand|
--- | --- |
1 | Peets |
2 | Folgers |
3 | Starbucks |
4 | Philz |
5 | Blue Bottle |
6 | Intelligentsia |
7 | Counter Culture |
8 | Stumptown |
9 | Groundworks |

After introducing the data in Part 3, we will focus on estimating the ranking-based model from
data by solving a linear program. Part 4 introduces this linear program and walks through solving
it with a fixed set of rankings. Part 5 then involves applying the column randomization approach,
while Part 6 involves using the column generation / constraint generation approach to solving the
problem. Finally, Part 7 looks at understanding the ranking model obtained from Part 6. A Jupyter
Notebook containing code snippets and skeleton code, HW2 - Coffee Code.ipynb, is provided to
help you with this problem.

**<font size="4">Part 1: Modeling preferences through rankings</font>**

a) For the same rankings given in the example, calculate the choice probabilities for the set of
products {3, 4}.

P(3 | {3, 4}) = $\lambda_2$ = 0.3

P(4 | {3, 4}) = $\lambda_1$ = 0.1

P(0 | {3, 4}) = $\lambda_3$ = 0.6

b) For the same rankings given in the example, calculate the choice probabilities for the set of
products {2, 3, 4}.

P(2 | {2, 3, 4}) = $\lambda_3$ = 0.6

P(3 | {2, 3, 4}) = $\lambda_2$ = 0.3

P(4 | {2, 3, 4}) = $\lambda_1$ = 0.1

P(0 | {2, 3, 4}) = 0

In [31]:
import numpy as np
import pandas as pd

**<font size="4">Part 2: Choice probabilities as a linear system of equations</font>**

a) For the M = 3 assortments in Part 1 (S1 = {1, 4, 5}, S2 = {3, 4}, S3 = {2, 3, 4}), and for the
K = 3 customer types (rankings) given in Part 1, write down the A matrix. Your matrix should
have 30 rows and 3 columns.
Hint: if you are having difficulty with this problem, take a look at the function permToA provided
in the skeleton code.

In [32]:
# Assortments
M = 3

# s = [[1, 4, 5, 0], 
#      [3, 4, 0], 
#      [2, 3, 4, 0]]

s = [[1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
     [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
     [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

# Customers
K = 3
N = 9 + 1

r = [[4, 3, 0, 1, 5, 6, 8, 7, 9, 2], 
     [3, 2, 4, 0, 1, 5, 6, 8, 7, 9], 
     [2, 5, 0, 1, 4, 3, 6, 8, 7, 9]]

p = [0.1, 0.3, 0.6]

##############################################################################################
def sToA(s, r):
     M = len(s)
     N = len(r[0])
     K = len(r)
     
     A = np.zeros((M*N, K))
     for k in range(K):
          for m in range(M):
               for i in range(N):
                    if s[m][i] == 1:
                         ele = np.where(np.array(s[m]) == 1)[0]
                         A[i + m*N][k] = np.where(all(r[k].index(i) <= r[k].index(e) for e in ele), 1, 0)
     return A
##############################################################################################

A = sToA(s, r)

b) Using this matrix, compute v = Aλ, where λ = (0.1, 0.3, 0.6). What do you get for v?

In [33]:
# Calculate dot product of A and p

v = np.dot(A, p)
v

array([0. , 0. , 0. , 0. , 0.4, 0.6, 0. , 0. , 0. , 0. , 0.6, 0. , 0. ,
       0.3, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0.3, 0.1, 0. ,
       0. , 0. , 0. , 0. ])

**<font size="4">Part 3: Understanding the data</font>**

In [34]:
# read data
assortments = pd.read_csv("coffee_assortments_v3.csv", header=None)
counts = pd.read_csv("coffee_transaction_counts_v3.csv", header=None)

# rename columns
assortments.columns = [1,2,3,4,5,6,7,8,9,0]
counts.columns = [1,2,3,4,5,6,7,8,9,0]

# rearrange columns
assortments = assortments[[0,1,2,3,4,5,6,7,8,9]]
counts = counts[[0,1,2,3,4,5,6,7,8,9]]

a) Which assortment has the most transactions?

In [35]:
ind = counts.sum(axis=1).idxmax()
list(assortments.loc[ind, assortments.loc[ind] == 1].index)

[0, 5, 7, 9]

b) Which assortment has the least transactions?

In [36]:
ind = counts.sum(axis=1).idxmin()
list(assortments.loc[ind, assortments.loc[ind] == 1].index)

[0, 2, 7]

c) How many assortments include the Starbucks brand?

In [37]:
len(assortments.loc[assortments[3] == 1])

18

Convert the matrix in coffee transaction counts.csv into an array of choice probabilities,
i.e., divide each row by its row sum. If you performed this step correctly, then the choice probability
of the no-purchase option for the first assortment should be 0.450413.

In [38]:
#divide each row with row sum
counts = counts.div(counts.sum(axis=1), axis=0)
counts.head()

,0,1,2,3,4,5,6,7,8,9
0,0.506887,0.0,0.399449,0.000000,0.000000,0.000000,0.000000,0.093664,0.000000,0.000000
1,0.419722,0.0,0.000000,0.163085,0.112516,0.145386,0.068268,0.091024,0.000000,0.000000
2,0.322196,0.0,0.275656,0.066826,0.125298,0.000000,0.065632,0.096659,0.000000,0.047733
3,0.297018,0.0,0.344037,0.000000,0.000000,0.152523,0.082569,0.000000,0.123853,0.000000
4,0.284424,0.0,0.256208,0.083521,0.109481,0.000000,0.066591,0.044018,0.120767,0.034989


d) For assortment m = 10, what fraction of customers purchased the Starbucks brand?

In [39]:
counts.loc[9, 3]

0.0

e) What fraction of customers offered assortment m = 3 chose to not purchase anything?

In [40]:
counts.loc[2, 0]

0.3221957040572792

**<font size="4">Part 4: Estimating the ranking-based model</font>**

a) Explain why solving problem EST is equivalent to solving the nonlinear problem EST-ℓ1. (Hint:
suppose that we have an optimal solution (λ, ε+, ε−) of problem EST. At optimality, what can
you say about what the values of ε+
m,i and ε−
m,i will be? Is it possible for both ε+
m,i and ε−
m,i to be
non-zero?)

Solving problem EST is equivalent to solving the nonlinear problem EST-ℓ1 because at optimality, 

b) Formulate and solve problem EST above for these six rankings. What is the optimal objective?
What is the optimal solution (λ, ε+, ε−)?

In [41]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *

##############################################################################################
# Initialize the model 
est = Model()

# Set the OutputFlag parameter to 0 to disable logging
est.Params.outputflag = 0

##############################################################################################
# Define Input Variables
M = 50
N = 9 + 1
K = 6
s = assortments.values.tolist()
r = [[1, 2, 7, 0, 6, 8, 9, 3, 5, 4], 
     [8, 9, 5, 7, 3, 6, 4, 1, 2, 0], 
     [2, 6, 9, 7, 4, 1, 5, 3, 0, 8], 
     [6, 3, 2, 1, 0, 5, 4, 7, 9, 8], 
     [5, 2, 4, 7, 8, 1, 6, 0, 3, 9], 
     [9, 2, 3, 7, 5, 1, 8, 4, 6, 0]]
v = counts.values.tolist()
A = sToA(s, r)

##############################################################################################
# Define Variables
e_plus = est.addVars(M, N, lb = 0)
e_minus = est.addVars(M, N, lb = 0)
p = est.addVars(K, lb = 0)

##############################################################################################
# Define Constraints
## Probability
est.addConstr(sum(p[k] for k in range(K)) == 1)

## EST
for m in range(M):
    for i in range(N):
        est.addConstr(sum((A[i + m*N][k] * p[k]) for k in range(K)) == v[m][i] + e_plus[m, i] - e_minus[m, i])

##############################################################################################
# Construct Objective
est.setObjective(sum((e_plus[m, i]) for m in range(M) for i in range(N)) + 
                 sum((e_minus[m, i]) for m in range(M) for i in range(N)), GRB.MINIMIZE)

##############################################################################################
# Update and solve
est.update()
est.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Objective Value:", est.objVal)
print("Optimal Solution:")
for k in range(K):
    print(f"p[{k}] = {p[k].X}")

##############################################################################################

Optimal Objective Value: 41.82943992668715
Optimal Solution:
p[0] = 0.24780157835400227
p[1] = 0.12518167609287636
p[2] = 0.05740700048172717
p[3] = 0.33337267199625975
p[4] = 0.18879181760068187
p[5] = 0.04744525547445255


**<font size="4">Part 5: A random sampling approach</font>**

a) Set your random seed to 200 and randomly generate 1000 rankings. Solve problem EST with
these 1000 rankings. What is your optimal objective value?

In [42]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *

##############################################################################################
# Initialize the model 
est = Model()

# Set the OutputFlag parameter to 0 to disable logging
est.Params.outputflag = 0

##############################################################################################
# Define Input Variables
M = 50
N = 9 + 1
K = 1000
s = assortments.values.tolist()

# Generate random permutations
np.random.seed(200)
r = []
for i in range(1000):
    r.append(np.random.permutation(10).tolist())

v = counts.values.tolist()
A = sToA(s, r)

##############################################################################################
# Define Variables
e_plus = est.addVars(M, N, lb = 0)
e_minus = est.addVars(M, N, lb = 0)
p = est.addVars(K, lb = 0)

##############################################################################################
# Define Constraints
## Probability
est.addConstr(sum(p[k] for k in range(K)) == 1)

## EST
for m in range(M):
    for i in range(N):
        est.addConstr(sum((A[i + m*N][k] * p[k]) for k in range(K)) == v[m][i] + e_plus[m, i] - e_minus[m, i])

##############################################################################################
# Construct Objective
est.setObjective(sum((e_plus[m, i]) for m in range(M) for i in range(N)) + 
                 sum((e_minus[m, i]) for m in range(M) for i in range(N)), GRB.MINIMIZE)

##############################################################################################
# Update and solve
est.update()
est.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Objective Value:", est.objVal)

##############################################################################################

Optimal Objective Value: 0.904453739014714


**<font size="4">Part 6: A column generation approach</font>**

a) How many possible rankings are there of the n + 1 options?

(n+1)!

b) Explain the meaning of the zi,j decision variables in problem SEP.

$z_{ij}$ is a binary decision variable that tells us the preference between $option_i$ and $option_j$.

If $z_{ij} = 1$, $option_i$ is more preferred than $option_j$

If $z_{ij} = 0$, $option_i$ is less or equally preferred than $option_j$

c) Explain the meaning of each constraint in problem SEP.

Constraint (13b): This makes sure that only one of $z_{ij}$ or $z_{ji}$ is 1. This makes sense because we know logically that if one of these conditions is true, the other MUST be false.


Constraint (13c): This makes sure that a particular option is not preferred over itself.


Constraint (13d): This makes sure that only two of $z_{ij}$ or $z_{jk}$ or $z_{ki}$ is 1. This makes sense because we know logically that if two of these conditions is true, the other MUST be false to maintain sequential ordering.


Constraint (13e): This makes sure that the most preferred option belongs to the options provided in the assortment.


Constraint (13f): This makes sure that if i is selected, it comes before option j for
that particular ranking for assortment m.


Constraint (13g): This makes sure that each assortment contains exactly one option chosen by
the customer. Essentially, every customer can choose exactly one option from each
assortment.


Constraint (13h): This makes sure that $z_{ij}$ is a binary decision variable equal to 1 if $option_i$ is more preferred than $option_j$


Constraint (13i): This makes sure that $A_{mi}$ is a binary decision variable equal to 1 if the option was chosen by the customer.

d) Explain how to determine the ranking σ from the values of the zi,j variables

We can reverse-engineer the decision variable $z_{ij}$ to determine σ. We can rank i for which sum of $z_{ij}$ is maximum in decsending order. The i with maximum sum is most preferred and the i with minimum sum is least preferred.

Implement problem SEP in Python, and create a function called separation that solves SEP
for two inputs, p val and q val, where p val is a numpy array corresponding to p, and q val is a
scalar variable corresponding to q. To check that you have implemented this function correctly, the
skeleton code provides example values of p val and q val together with what the optimal objective
value of SEP should be for those inputs. (In addition, ensure that your constraint generation
procedure saves the ranking corresponding to each solution of SEP, as these will be needed for Part
7.)
Once you have implemented the separation function, implement the constraint generation
procedure. Skeleton code for the procedure is provided in the accompanying Jupyter Notebook.
Solve problem EST-DUAL using the constraint generation procedure. Use the six rankings provided
in Part 4 as the initial rankings that your constraint generation procedure starts from.

In [43]:
# =================================================
# First, set up your dual problem here:
# =================================================

##############################################################################################
#Importing necessary libraries
from gurobipy import *

##############################################################################################
# Initialize the model
m_dual = Model()

# Set the OutputFlag parameter to 0 to disable logging
m_dual.Params.outputflag = 0

##############################################################################################
# Define Input Variables
M = 50
N = 9 + 1
K = 6
s = assortments.values.tolist()
r = [[1, 2, 7, 0, 6, 8, 9, 3, 5, 4], 
     [8, 9, 5, 7, 3, 6, 4, 1, 2, 0], 
     [2, 6, 9, 7, 4, 1, 5, 3, 0, 8], 
     [6, 3, 2, 1, 0, 5, 4, 7, 9, 8], 
     [5, 2, 4, 7, 8, 1, 6, 0, 3, 9], 
     [9, 2, 3, 7, 5, 1, 8, 4, 6, 0]]
v = counts.values.tolist()
A_list = sToA(s, r)

##############################################################################################
# Define Variables
p = m_dual.addVars(M, N, lb = -1, ub = 1) 
q = m_dual.addVar()

##############################################################################################
# Add constraints
for k in range(K):
    m_dual.addConstr(sum(A_list[i + m*N, k] * p[m, i] for i in range(N) for m in range(M)) + q <= 0)
# (bounds on p_{m,i}, and constraints corresponding to 
# initial six rankings.)
# You may find it helpful to maintain a list, A_list, which
# contains the 2D arrays generated by permToA. 
# You may also find it helpful to maintain a list, ranking_list,
# which contains the rankings corresponding to A_list. 

##############################################################################################
# Add objective function
m_dual.setObjective(sum(v[m][i] * p[m, i] for i in range(N) for m in range(M)) + q, GRB.MAXIMIZE)

##############################################################################################
# Update and solve
m_dual.update()
m_dual.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Objective Value:", m_dual.objVal)

##############################################################################################

Optimal Objective Value: 41.82943992668715


In [44]:
# =================================================
# Set up your separation subproblem in Gurobi here: 
# =================================================

##############################################################################################
#Importing necessary libraries
from gurobipy import *

##############################################################################################
# Initialize model
m_sub = Model()

# Set the OutputFlag parameter to 0 to disable logging
m_sub.Params.OutputFlag = 0

##############################################################################################
# Define Variables
A = m_sub.addVars(M, N, vtype = GRB.BINARY)
z = m_sub.addVars(N, N, vtype = GRB.BINARY)

##############################################################################################
# Add constraints
for i in range(N):
    for j in range(N):
        if (i != j):
            m_sub.addConstr(z[i, j] + z[j, i] == 1)
        else:
            m_sub.addConstr(z[i, j] == 0)

for i in range(N):
    for j in range(N):
        for k in range(N):
            if (i != j and j != k and i != k):
                m_sub.addConstr(z[i, j] + z[j, k] + z[k, i] <= 2)

for m in range(M):
    for i in range(N):
        if (assortments.iloc[m, i] == 0):
            m_sub.addConstr(A[m, i] == 0)
            
for m in range(M):
    for i in range(N):
        for j in range(N):
            if (i != j and assortments.iloc[m, i] == 1 and assortments.iloc[m, j] == 1):
                m_sub.addConstr(A[m, i] <= z[i, j])

for m in range(M):
    m_sub.addConstr(sum(A[m, i] for i in range(N)) == 1)

##############################################################################################
# Update and solve
m_sub.update()

##############################################################################################


In [45]:
# =====================================
# Set up your separation function here:
# =====================================


def separation(p_val, q_val):
    m_sub.setObjective(sum(p_val[m, i]*A[m, i] for m in range(M) for i in range(N)) + q_val, GRB.MAXIMIZE)
    
    m_sub.update()
    m_sub.optimize()
    
    A_val = np.asarray([[A[m,i].x for i in range(N)] for m in range(M)])
    z_val = np.asarray([[z[i,j].x for j in range(N)] for i in range(N)])
    
    # Use z_val to determine the ranking:
    ranking = []
    row_sums = np.sum(z_val, axis = 1)
    
    for i in range(N):
        index = np.where(row_sums == 9-i)[0][0]
        ranking.append(index+1)
    
    # Retrieve objective value
    sub_obj = m_sub.objVal
    
    return sub_obj, A_val, ranking

In [46]:
# =======================================================
# Test your separation function below:
# =======================================================

np.random.seed(10)
p_val = np.random.randint(-1,1,size = (M,N))
q_val = 0.0

sub_objval, single_A, single_perm = separation(p_val, q_val)

print(sub_objval) # Should display -13.0
print(single_perm) # Should display [8 2 5 0 4 3 1 9 7 6]

GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [47]:
# =======================================================
# Finally, put your constraint generation procedure here.
# =======================================================

m_dual.Params.OutputFlag = 0
m_dual.update()
m_dual.optimize()

dual_obj = m_dual.objval
CG_iter = 0
print("Iteration ", CG_iter, ": ", dual_obj)

p_val = np.asarray( [ [p[i,j].x for j in range(N)] for i in range(M)] )
q_val = q.x

# Solve separation problem for the first time:
sub_objval, single_A, single_ranking = separation(p_val, q_val) 

# Create the flag isOptimal to decide whether the solution is optimal
# or not (does it satisfy all the dual constraints or not):
tol_eps = 1e-5
isOptimal = (sub_objval <= 0 + tol_eps)

while (not isOptimal):
    # Add the column of single_A to A_list:
    A_list.append(single_A)
    
    # Add the ranking of single_ranking to r:
    r = np.append(r, [single_ranking], axis = 0)
    
    # Add constraint corresponding to this single_A / single_ranking
    # to dual:
    m_dual.addConstr(sum(A_list[-1][m][n]*p[m, n] for m in range(M) for n in range(N)) + q <= 0)
    
    # Update and re-solve the dual:
    m_dual.update()
    m_dual.optimize()
    
    # Retrieve values of dual variables p and q, and dual objective
    # value: 
    dual_obj = m_dual.objVal
    p_val = np.asarray( [ [p[i,j].x for j in range(N)] for i in range(M)] )
    q_val = q.x
    
    # Print information about this iteration:
    CG_iter += 1 
    print()
    print("Iteration ", CG_iter, ": ", dual_obj)
    
    # Solve separation problem, and update the isOptimal flag.
    sub_objval, single_A, single_ranking = separation(p_val, q_val) 
    isOptimal = (sub_objval <= 0 + tol_eps)
    print("sub_objval = ", sub_objval)
    print("isOptimal = ", isOptimal)

Iteration  0 :  41.82943992668715


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

e) What is the optimal objective value?

In [48]:
sub_objval

NameError: name 'sub_objval' is not defined

f) How many rankings are generated by your constraint generation procedure?

In [49]:
len(A_list)

6

**<font size="4">Part 7: Understanding the column generation solution</font>** -- Couldn't attempt

a) How many rankings have a non-zero value of λk associated with them?

b) What are the top three rankings in terms of λk? For these three rankings, which are the top
three most preferred options?

c) What is the average rank of each of the ten options (the n = 9 products plus the no-purchase
option), where the average is taken with respect to the rankings and the probability distribution
λ in your constraint generation solution? (Lower rank values correspond to more preferred
options.)

d) Suppose that we offer the assortment S = {2, 4, 5, 6}. This corresponds to offering Folgers, Philz,
Blue Bottle and Intelligentsia. What are the predicted choice probabilities for the products in
this assortment? What is the predicted probability of customers not purchasing anything?


**<font size="5">2. Designing a Sushi Menu</font>**

In [2]:
util = pd.read_csv("sushi_utilities_mat.csv", header=None)
info = pd.read_csv("sushi_info.csv", header=None)
info.columns = ['sushi', 'category', 'price']
desc = pd.read_csv("sushi_descriptions.csv", header = None, index_col = False).drop(0, axis = 1)
desc.columns = ['sushi']
desc['desc'] = desc['sushi'].str.extract(r'\((.*?)\)')
desc['sushi'] = desc['sushi'].str.split(' ').str[0]

util.columns = info['sushi']

**<font size="4">Part 1: Understanding the data</font>**

a) What are the five most preferred sushis for customer 1? Hint: use the command argsort from
numpy on a particular row of the utility matrix.

In [3]:
answer = (util.iloc[0]).sort_values(ascending = False).head().index
for ans in answer:
    print(ans)

negi_toro
ika
maguro
samon
inari


b) What are the five least preferred sushis for customer 2?

In [4]:
answer = (util.iloc[1]).sort_values(ascending = True).head(5).index
for ans in answer:
    print(ans)

karasumi
komochi_konbu
kyabia
awabi
kazunoko


c) For each customer, compute the rank of each of the 100 sushis according to their utilities. Which sushis are the top five, i.e., the five with the best average rank over the 500 customers? What do you notice about the sushi (specifically, the type of fish)?

In [16]:
a = np.argsort(np.argsort(np.argsort(util, axis=1), axis=1).mean(axis = 0))
for key, value in enumerate(a):
    if value in [0, 1, 2, 3, 4]:
        print(value, info.iloc[key]['sushi'])

4 sarada_maki
1 mekabu
3 kujira
0 karei
2 shishamo


d) Which sushi has the worst average rank over the 500 customers?

In [256]:
a = np.argsort(np.argsort(util, axis = 1).mean(axis = 0))
for key, value in enumerate(a):
    if value in [99]:
        print(info.iloc[key]['sushi'])

akagai


e) Which sushi is the most controversial, as measured by the standard deviation of its rank over the 500 customers?

In [257]:
a = np.argsort(np.argsort(util, axis = 1).var(axis = 0))
for key, value in enumerate(a):
    if value in [0]:
        print(info.iloc[key]['sushi'])

tai


**<font size="4">Part 2: Common-sense solutions</font>**

In [258]:
util.columns = list(range(1,101))

In [259]:
assortment = [1, 2, 3, 4, 5]

def pick(assortment, no_purchase = 3):
    df = util[assortment]
    df = df[df > no_purchase]
    df = pd.DataFrame(df.idxmax(axis = 1).value_counts())
    df.index = df.index - 1
    df = pd.merge(df, info.dropna(), how = 'left', left_index = True, right_index = True)
    df['revenue'] = df['price'] * df['count']
    df = df.revenue.sum()/len(util)
    return df

pick(assortment)

14.239619999999999

a) Suppose that we simply offer all sushi products, that is, we set S = {1, 2, . . . , n}. What is the
expected revenue in this case?

In [260]:
assortment = list(range(1, 101))
pick(assortment)

21.509720000000005

b) Suppose that we offer the ten highest revenue sushis, that is, we set S = {i1, . . . , i10}, where
ri1 ≥ ri2 ≥ · · · ≥ rin . Which are the ten highest revenue sushis? What is the expected revenue
in this case?

In [261]:
assortment = list(info['price'].sort_values(ascending = False).head(10).index + 1)
pick(assortment)

25.63944

c) Suppose that for every customer k, we determine his/her most preferred sushi, i∗
k = arg max1≤i≤n uk,i.
We then offer all of the most preferred sushis, that is, we set S = {i∗
1, i∗
2, . . . , i∗
K }. (Note that
some sushis may be the top choice of more than one customer.)
What is the expected revenue in this case? What do you notice about this value?

In [262]:
assortment = list(util.idxmax(axis = 1).value_counts().index)
pick(assortment)

21.509720000000005

I observe that this value is exactly equal to when we offer all sushi products and each customer picks their highest utility sushi. (answer in part a)

d) Explain why the solution in (a) may be suboptimal.

The solution in (a) may be suboptimal because customers choose their highest utility sushi, ut that doesn't necessarily translate into higher revenue for the restaurant. We want to design a menu that will offer customers higher utility than the no-purchase option, and also generate high revenue for the resturant. There needs to be a perfect balance of utility and revenue.

e) Explain why the solution in (b) may be suboptimal.

The solution in (b) may be suboptimal because restaurant chooses their highest revenue sushis, but that doesn't necessarily mean the customers buy those sushis due to their utility. We want to design a menu that will offer customers higher utility than the no-purchase option, and also generate high revenue for the resturant. There needs to be a perfect balance of utility and revenue.

**<font size="4">Part 3: An integer optimization model</font>**

a) Explain how constraints (14b) – (14e) correctly model the preferences of each customer.

Constraint (14b): This makes sure that every customer can have exactly one most preferred sushi.

Constraint (14c): This makes sure that the utility of the most preferred sushi for each customer greater than or equal to the utility of any other sushi in the assortment offered (including the no purchase option)

Constraint (14d): This makes sure that the utility of the most preferred sushi for customer must be greater
than or equal to the utility of not purchasing any sushi

Constraint (14e): This makes sures that the most preferred option is included in the offered
assortment.

b) Implement the linear optimization relaxation of the above problem in Python. The relaxation is
obtained by relaxing x and y to be continuous decision variables as opposed to binary decision
variables – that is, we replace constraints (14f) and (14g) with:
0 ≤ xi ≤ 1, ∀ i ∈ {1, . . . , n}, (15)
0 ≤ yk,i ≤ 1, ∀ k ∈ {1, . . . , K}, i ∈ {1, . . . , n}. (16)
What is the optimal objective value of the relaxation?

In [269]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *

##############################################################################################
# Initialize the model 
relax = Model()

# Set the OutputFlag parameter to 0 to disable logging
relax.Params.outputflag = 0

##############################################################################################
# Define Input Variables
K = 500
N = 100
u = util.copy()
u[0] = 3 # outside option
u = u[list(range(101))]
info = np.insert(info, 0, ('no purchase', -1, 0), axis = 0)

##############################################################################################
# Define Variables
y = relax.addVars(K, N+1, lb = 0, ub = 1, vtype = GRB.CONTINUOUS)
x = relax.addVars(N+1, lb = 0, ub = 1, vtype = GRB.CONTINUOUS)

##############################################################################################
# Define Constraints
for k in range(K):
    relax.addConstr(sum(y[k, i] for i in range(N+1)) == 1)

for k in range(K):
    for i in range(1, N+1):
        relax.addConstr(sum(u.iloc[k, j]*y[k, j] for j in range(N+1)) >= u.iloc[k, i]*x[i-1] + u.iloc[k, 0]*(1 - x[i-1]))
 
for k in range(K):
    relax.addConstr(sum(u.iloc[k, j]*y[k, j] for j in range(N+1)) >= u.iloc[k, 0])

for k in range(K):
    for i in range(1, N+1):
        relax.addConstr(y[k, i] <= x[i-1])

##############################################################################################
# Construct Objective
relax.setObjective(sum((info.price[i-1]*y[k, i]) for i in range(1, N+1) for k in range(K))/K, GRB.MAXIMIZE)

##############################################################################################
# Update and solve
relax.update()
relax.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Objective Value:", relax.objVal)

##############################################################################################

KeyboardInterrupt: 

c) A manager claims that it should be possible to obtain an assortment with an expected per customer revenue of $32. Explain why this is impossible based on your answer to (b).

We achieve an optimal solution of $30.59 in our linear relaxation. This means we cannot obtain an expected per customer revenue of more than $30.59 without violating at least one of the constraints.

d) Now, implement the integer version of the above problem (i.e., the binary constraints (14f) and
(14g) are enforced) in Python. Solve the problem. What is the expected per-customer revenue
of the optimal assortment? How much does this improve over the assortments from Part 2?

In [ ]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *

##############################################################################################
# Initialize the model 
relax = Model()

# Set the OutputFlag parameter to 0 to disable logging
relax.Params.outputflag = 0

##############################################################################################
# Define Input Variables
K = 500
N = 100
u = util.copy()
u[0] = 3 # outside option
u = u[list(range(101))]
info = np.insert(info, 0, ('no purchase', -1, 0), axis = 0)

##############################################################################################
# Define Variables
y = relax.addVars(K, N+1, lb = 0, ub = 1, vtype = GRB.BINARY)
x = relax.addVars(N+1, lb = 0, ub = 1, vtype = GRB.BINARY)

##############################################################################################
# Define Constraints
for k in range(K):
    relax.addConstr(sum(y[k, i] for i in range(N+1)) == 1)

for k in range(K):
    for i in range(1, N+1):
        relax.addConstr(sum(u.iloc[k, j]*y[k, j] for j in range(N+1)) >= u.iloc[k, i]*x[i-1] + u.iloc[k, 0]*(1 - x[i-1]))
 
for k in range(K):
    relax.addConstr(sum(u.iloc[k, j]*y[k, j] for j in range(N+1)) >= u.iloc[k, 0])

for k in range(K):
    for i in range(1, N+1):
        relax.addConstr(y[k, i] <= x[i-1])

##############################################################################################
# Construct Objective
relax.setObjective(sum((info.price[i-1]*y[k, i]) for i in range(1, N+1) for k in range(K))/K, GRB.MAXIMIZE)

##############################################################################################
# Update and solve
relax.update()
relax.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Objective Value:", relax.objVal)

##############################################################################################

e) What is the optimal set of sushis the restaurant should offer?